Import the required modules.

In [1]:
import re
import numpy as np
import Levenshtein as levenshtein
import json
import fuzzy
import numpy as np
from sklearn.model_selection import train_test_split
import soundex

Read all files and store into lists.

In [2]:
def readfile(filename):
    f = open(filename, "r")
    lines = f.readlines()
    ls = []
    for line in lines:
        ls.append(line.strip())
    f.close()
    return ls

In [3]:
dict_ls = readfile("dict.txt")
correct_ls = readfile("wiki_correct.txt")
misspell_ls = readfile("wiki_misspell.txt")

Find the best match(es) for the word in dictionary according to the global edit distance.

In [4]:
def best_match_levenshtein(target):
    # init best matches
    min_dist = levenshtein.distance(target,dict_ls[0])
    best_matches = [dict_ls[0]]
    
    for word in dict_ls[1:]:
        if abs(len(word) - len(target)) > min_dist:  # not possible to be min_dist, skip
            continue
        dist = levenshtein.distance(target,word)  # cal global edit distance
        # replace if shorter distance
        if dist < min_dist:
            min_dist = dist
            best_matches = [word]
        elif dist == min_dist:
            best_matches.append(word)
    
    return best_matches

Find the best matches for the words in wiki data sets according to levenshtein.

In [ ]:
levenshtein_correction_dict = {}
for word in wiki_misspell_ls:
    if word not in levenshtein_correction_dict: # avoid repeated word
        levenshtein_correction_dict[word] = best_match_levenshtein(word)

Save correction correction results.

In [ ]:
with open('levenshtein_correction.json', 'w') as fp:
    json.dump(levenshtein_correction_dict, fp)
    fp.close()

In [5]:
with open('levenshtein_correction.json', 'r') as fp:
    levenshtein_correction_dict = json.load(fp)
    fp.close()

Save the random selected mispelled subset (1335 tokens).

In [49]:
misspell_subset, misspell_left, correct_subset, correct_left = train_test_split(misspell_ls, 
                                                                                correct_ls,
                                                                                train_size = 0.3,
                                                                                test_size = 0.7,
                                                                                shuffle = True)

Combine two subsets into a list and save as json file

In [68]:
combined_subset = [(misspell_subset), (correct_subset)]

In [69]:
with open('misspell_correct_subset.json', 'w') as fp:
    json.dump(combined_subset, fp)
    fp.close()

In [70]:
with open('misspell_correct_subset.json', 'r') as fp:
    combined_subset = json.load(fp)
    fp.close()

In [71]:
mispell_subset = combined_subset[0]
correct_subset = combined_subset[1]

Find the best matches for the words in wiki data subset according to levenshtein.

In [73]:
subcorrection_dict = {}
for word in misspell_subset:
    if word not in subcorrection_dict: # avoid repeated word
        subcorrection_dict[word] = best_match_levenshtein(word)

neigbour
neighbour


KeyboardInterrupt: 

Save correction correction results.

In [ ]:
with open('ged_sub_correction.json', 'w') as fp:
    json.dump(sub_correction_dict, fp)
    fp.close()

In [ ]:
with open('ged_sub_correction.json', 'r') as fp:
    sub_correction_dict = json.load(fp)
    fp.close()

Evaluation Metric Implementation (Precision and Recall and F-Score)

In [ ]:
print len

In [6]:
def wiki_eval(result_dict, curr_misspell_ls, curr_correct_ls):
    tp = 0
    fp = 0
    fn = 0

    for i in range(len(curr_misspell_ls)):
        word = curr_misspell_ls[i]
        true_word = curr_correct_ls[i]
        if true_word in result_dict[word]:
            tp += 1
            fp += len(result_dict[word]) - 1
        else:
            fn += 1 # miss
            fp += len(result_dict[word])
            
    precision = tp*1.0 / (tp + fp)
    recall = tp*1.0 / (tp + fn)
    fscore = 2*(precision*recall)/(precision + recall)

    return precision,recall,fscore

Precision, recall, and F-score for levenshtein (baseline)

In [10]:
print wiki_eval(levenshtein_correction_dict, misspell_ls, correct_ls)
# print len(wiki_misspell_ls)

(0.2604320804971885, 0.7904783292162587, 0.3917858534142134)


GED implementation with customized parameter set

In [ ]:
# transform s1 to s2# trans 
def edit_distance(s1,s2, params):
    # Levenshteiin Distance (match,insert,delete,replace): the lower the better
    (m,i,d,r) = params
    
    # init matrix
    s1_len = len(s1)
    s2_len = len(s2)
    A = np.zeros((s2_len+1,s1_len+1))
    A[0][0] = 0
    for j in range(1, s2_len+1):
        A[j][0] = j*i;  # insert
    for k in range(1, s1_len+1):
        A[0][k] = k*d;  # delete
        
    # filling in table
    for j in range(1,s2_len+1):
        for k in range(1,s1_len+1):
            A[j][k] = min(A[j][k-1] + d,
                          A[j-1][k] + i,
                          A[j-1][k-1] + equal(s1[k-1],s2[j-1],m,r))
    return A[s2_len,s1_len]
    
def equal(ch1,ch2,cost1,cost2):
    if ch1 == ch2:
        return cost1
    else:
        return cost2

Testing implementation

In [ ]:
edit_distance("crat","arts",(0,1,1,1))

Find the best match(es) for the word in dictionary according to the soundex table.

In [ ]:
def best_match_soundex(target):
    
    soundex = fuzzy.Soundex(4)
    target_soundex = soundex(target)
    
    best_matches = []
    
    for word in dict_ls:
        print word
        print soundex(word)
        if (soundex(word) == target_soundex):  # same soundex
            best_matches.append(word)
    print best_matches
    
    return best_matches

In [ ]:
wiki_correction_dict_soundex = {}
for word in wiki_misspell_ls:
    if word not in wiki_correction_dict_soundex:
        wiki_correction_dict_soundex[word] = best_match_soundex(word)

In [ ]:
soundex = fuzzy.Soundex(4)
soundex("aam")

In [ ]:
print wiki_dict_ls